## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [2]:
NAME = "Christina Koutsi"
AEM = "9871"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [3]:
# Always run this cell
import numpy as np
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [4]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7faa51bc0990>)

In [5]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

ge f-measure weighted and balanced accuracy of your models. You ## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [6]:
# BEGIN CODE HERE

cls1 = "DTC" # Classifier #1
cls2 = "LR" # Classifier #2
cls3 = "KNN" # Classifier #3

estimator = []
estimator.append((cls1, DecisionTreeClassifier()))
estimator.append((cls2, LogisticRegression(solver='lbfgs',
                                     multi_class='multinomial',
                                     max_iter=200)))
estimator.append((cls3, KNeighborsClassifier(n_neighbors=5)))

soft_vcls = VotingClassifier(estimators=estimator, voting='soft') # Voting Classifier
hard_vcls = VotingClassifier(estimators=estimator, voting='hard') # Voting Classifier

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

scoresSoft = cross_validate(soft_vcls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

s_avg_fmeasure = np.mean(scoresSoft['test_f1_score']) # The average f-measure
s_avg_accuracy = np.mean(scoresSoft['test_accuracy']) # The average accuracy

scoresHard = cross_validate(soft_vcls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

h_avg_fmeasure = np.mean(scoresHard['test_f1_score']) # The average f-measure
h_avg_accuracy = np.mean(scoresHard['test_accuracy']) # The average accuracy

#END CODE HERE

In [7]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DTC', DecisionTreeClassifier()),
                             ('LR',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial')),
                             ('KNN', KNeighborsClassifier())],
                 voting='soft')
F1 Weighted-Score: 0.8533 & Balanced Accuracy: 0.826


You should achive above 82% (Soft Voting Classifier)

In [8]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DTC', DecisionTreeClassifier()),
                             ('LR',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial')),
                             ('KNN', KNeighborsClassifier())])
F1 Weighted-Score: 0.852 & Balanced Accuracy: 0.8248


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [9]:
# BEGIN CODE HERE

cls1 = MLPClassifier(activation="relu", alpha=0.1, hidden_layer_sizes=(10, 10, 10),
                     learning_rate="constant", max_iter=1000, random_state=RANDOM_STATE)

cls2 = RandomForestClassifier(n_estimators=150, criterion="gini", max_depth=12,
                              max_features="auto", min_samples_split=0.050, n_jobs=-1, 
                              random_state=RANDOM_STATE)

cls3 = SVC(C=50, degree=1, gamma="auto", kernel="rbf", probability=False)

classifiers = [('mlp', cls1),
               ('rfc', cls2),
               ('svc', cls3)]

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

scls = StackingClassifier(estimators=classifiers,
                          final_estimator=LogisticRegression(),
                          stack_method='auto',
                          n_jobs=-1,
                          passthrough=False)

scores = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

avg_fmeasure = np.mean(scores['test_f1_score'])  # The average f-measure
avg_accuracy = np.mean(scores['test_accuracy'])  # The average accuracy

#END CODE HERE

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [10]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('mlp',
                                MLPClassifier(alpha=0.1,
                                              hidden_layer_sizes=(10, 10, 10),
                                              max_iter=1000, random_state=42)),
                               ('rfc',
                                RandomForestClassifier(max_depth=12,
                                                       min_samples_split=0.05,
                                                       n_estimators=150,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('svc', SVC(C=50, degree=1, gamma='auto'))],
                   final_estimator=LogisticRegression(), n_jobs=-1)
F1 Weighted Score: 0.8876 & Balanced Accuracy: 0.8663


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [11]:
# BEGIN CODE HERE

ens1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE)
ens2 = BaggingClassifier(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE)
ens3 = AdaBoostClassifier(n_estimators=100, random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

scores = cross_validate(tree, X, y, cv=kfold, scoring=scoring, n_jobs=-1)

f_measures["Decision tree"] = scores['test_f1_score']
accuracies["Decision tree"] = scores['test_accuracy']

scores = cross_validate(ens1, X, y, cv=kfold, scoring=scoring)

f_measures["Random forest"] = scores['test_f1_score']
accuracies["Random forest"] = scores['test_accuracy']

scores = cross_validate(ens2, X, y, cv=kfold, scoring=scoring)

f_measures["Bagging"] = scores['test_f1_score']
accuracies["Bagging"] = scores['test_accuracy']

scores = cross_validate(ens3, X, y, cv=kfold, scoring=scoring, n_jobs=-1)

f_measures["Ada boost"] = scores['test_f1_score']
accuracies["Ada boost"] = scores['test_accuracy']

#END CODE HERE

In [12]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,np.round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,np.round(score,4)))

RandomForestClassifier(n_jobs=-1, random_state=42)
BaggingClassifier(n_estimators=100, n_jobs=-1, random_state=42)
AdaBoostClassifier(n_estimators=100, random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier:Decision tree -  F1 Weighted:[0.7354 0.7602 0.7553 0.7578 0.7658 0.7313 0.7537 0.7531 0.7447 0.7638]
Classifier:Random forest -  F1 Weighted:[0.839  0.8619 0.8423 0.855  0.866  0.8312 0.861  0.8369 0.8297 0.8762]
Classifier:Bagging -  F1 Weighted:[0.8404 0.8502 0.856  0.8464 0.8452 0.8321 0.8543 0.8409 0.8176 0.8771]
Classifier:Ada boost -  F1 Weighted:[0.7983 0.8251 0.8353 0.8445 0.8333 0.8217 0.8421 0.8228 0.8302 0.8534]
Classifier:Decision tree -  BalancedAccuracy:[0.6881 0.7071 0.7238 0.7048 0.7262 0.6833 0.7208 0.7184 0.7136 0.7136]
Classifier:Random forest -  BalancedAccuracy:[0.8071 0.8214 0.8048 0.8119 0.8357 0.7833 0.8282 0.7971 0.7971 0.8401]
Classifier:Bagging -  BalancedAccuracy:[0.8119 0.8095 0.8262 0.8048 0.8143 0.7905 0.8234 0.8067 0.7828 0.8449]
Classifi

**2.2** Describe your classifiers and your results.

In this exercise I'm using ensembles of decision trees and each uses one of the following methods for producing homogeneous ensembles: the Random Forest Classifier, the Bagging Classifier and the AdaBoost Classifier. I'm also using a simple Decision Tree Classifier.

As shown above, every ensemble is better than a simple Decision Tree Classifier in achieving better f1-scores and balanced-accuracy.

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

One solution is to use parallelisation. Specifically, the Bagging Classifier is can easily be parallelised, as it uses the same training set as the starting data.


## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [13]:
# BEGIN CODE HERE
cls1 = MLPClassifier(activation="relu", alpha=0.1, hidden_layer_sizes=(10, 10, 10),
                     learning_rate="constant", max_iter=1000, random_state=RANDOM_STATE)

cls2 = RandomForestClassifier(n_estimators=150, criterion="gini", max_depth=10,
                              max_features="auto", min_samples_split=0.005, n_jobs=-1, 
                              random_state=RANDOM_STATE)

cls3 = SVC(C=50, degree=1, gamma="auto", kernel="rbf", probability=False)

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

classifiers = [('mlp', cls1),
               ('rfc', cls2),
               ('svc', cls3)]
               
best_cls = StackingClassifier(estimators=classifiers,
                          final_estimator=LogisticRegression(),
                          stack_method='auto',
                          n_jobs=-1,
                          passthrough=False)

results = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

best_fmeasure = results["test_f1_score"]
best_accuracy = results["test_accuracy"]
#END CODE HERE

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [14]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:[0.88381743 0.86836935 0.88531187 0.89534884 0.91170431 0.87843137
 0.90725806 0.87169043 0.862423   0.91119691] & Balanced Accuracy:[0.86666667 0.84047619 0.86428571 0.87142857 0.89761905 0.85238095
 0.8902148  0.849642   0.84009547 0.8902148 ]


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

Classifier:
F1 Weighted-Score:[0.87025948 0.8984375  0.892      0.91119691 0.89513109 0.89156627
 0.85224839 0.8902439  0.85835095 0.88047809] & Balanced Accuracy:[0.8452381  0.87619048 0.87142857 0.89047619 0.86666667 0.87142857
 0.8353222  0.87112172 0.84009547 0.85680191]

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [15]:
# BEGIN CODE HERE
cls = RandomForestClassifier(n_estimators=150, n_jobs=-1, random_state=RANDOM_STATE)
cls.fit(X,y)
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!